In [1]:
from peft import PeftModel, PeftConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

/home/ysx/miniconda3/envs/llama/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
peft_model_id = "/home/ysx/src/AI/llm_demo/outputs/zypt_llama_7B"
config = PeftConfig.from_pretrained(peft_model_id)


In [5]:

inference_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_4bit=True, 
    device_map="auto",
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:22<00:00, 11.46s/it]


In [6]:
tokenizer = AutoTokenizer.from_pretrained(
    config.base_model_name_or_path,
)

In [7]:
model = PeftModel.from_pretrained(inference_model, peft_model_id, device_map="auto")

In [8]:
model = model.eval()

In [8]:
def generate_prompt(instruction, input):
    return (
        "Below is an instruction that describes a task, paired with an input that provides further context. "
        "Write a response that appropriately completes the request.\n\n"
        f"### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n"
    )
   

In [9]:
def make_inference(instruction, input):
  question = generate_prompt(instruction, input)
  batch = tokenizer(question, return_tensors='pt')

  with torch.cuda.amp.autocast():
    output_tokens = model.generate(**batch, max_new_tokens=200)

  print(tokenizer.decode(output_tokens[0], skip_special_tokens=True))

In [9]:

def make_inference(text):
  batch = tokenizer(text, return_tensors='pt')

  with torch.cuda.amp.autocast():
    output_tokens = model.generate(**batch, max_new_tokens=300)

  print(tokenizer.decode(output_tokens[0], skip_special_tokens=True))

In [16]:
t = "什么是中医治疗八法中的和法？"

In [17]:
make_inference(t)

什么是中医治疗八法中的和法？和法是调整阴阳，恢复五行的平衡，纠正人体与自然界的关系的一种治法。其主要内容包括：寒热、虚实、表里、气血、阴阳。和法的调整目的是使“正”发挥正的作用，使“邪”发挥邪的作用。“正”包括寒、热、虚、实四个方面。“邪”也包括寒、热、虚、实四个方面。在疾病发生发展过程中，正与邪总是相互制约，相互斗争，正胜邪退，邪胜正退。所以，和法是以扶正为主，即以扶正药为主，用之在疾病发展过程中，扶正药，以使正气发生作用，以使正气发挥正气的作用。


In [13]:
make_inference(i, t)

/home/ysx/miniconda3/envs/llama/lib/python3.11/site-packages/transformers/generation/utils.py:1636: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
/home/ysx/miniconda3/envs/llama/lib/python3.11/site-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
解释五行相生和相克之间的关系。

### Input:
五行相生和相克是五行理论中的两个基本概念。相生是指五行之间的相互促进和滋养，例如木生火表示木能够促进火的生成。相克则是五行之间的相互制约和克制，例如木克土表示木能够制约土的过度发展。这两种关系在五行理论中是相辅相成的，相生关系保证了五行之间的正向发展，而相克关系则是一种负向调节，防止任何一行的过度发展而破坏整体平衡。

### Response:
五行相生和相克之间是相互依存的，没有相生就没有相克，没有相克也就没有相生。五行之间通过相生关系来促进和滋养，以维持五行之间的平衡和协调。例如，木生火，火生土，土生金，金生水，水生木。这五个木的顺序，就是五行的顺序，即木、火、土、金、水。五行的顺序就是由木生火，火生土，土生金，金生水，水生木所决定的。如果木不生火，火不生土，土不生金，金不生水，水不生木，这五个木就不能构成五行，也就不能构成五行的顺序，也就不能构成五行的平衡。五行之间的平衡和协调，就是通过相生关系来维持的。


In [14]:
# merge base model and lora model as a standalone model.
# merged_model = model.merge_and_unload()

In [15]:
tokenizer("111111</s>")

{'input_ids': [1, 29871, 29896, 29896, 29896, 29896, 29896, 29896, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}